# chat bot api

In [1]:
from chatbot import Chat,reflections,multiFunctionCall
import wikipedia
import os

# Wikipedia API connection

In [2]:
def whoIs(query,sessionID="general"):
    print(query)
    try:
        return wikipedia.summary(query)
    except:
        for newquery in wikipedia.search(query):
            try:
                return wikipedia.summary(newquery)
            except:
                pass
    return "I don't know about "+query

# Emotion Detector Connection

In [3]:
from keras.preprocessing.image import img_to_array
import imutils
import cv2
from keras.models import load_model
import numpy as np
import playsound
# parameters for loading data and images
detection_model_path = 'Emotion/haarcascade_files/haarcascade_frontalface_default.xml'
emotion_model_path = 'Emotion/models/_mini_XCEPTION.102-0.66.hdf5'

# hyper-parameters for bounding boxes shape
# loading models
face_detection = cv2.CascadeClassifier(detection_model_path)
emotion_classifier = load_model(emotion_model_path, compile=False)
EMOTIONS = ["angry" ,"disgust","scared", "happy", "sad", "surprised",
 "neutral"]

def emo(query,sessionID="general"):
    cv2.namedWindow('your_face')
    camera = cv2.VideoCapture(0)

    while True:
        frame = camera.read()[1]
        frame = imutils.resize(frame,width=300)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_detection.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(30,30),flags=cv2.CASCADE_SCALE_IMAGE)
        canvas = np.zeros((250, 300, 3), dtype="uint8")
        frameClone = frame.copy()
        if len(faces) > 0:
            faces = sorted(faces, reverse=True,
            key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))[0]
            (fX, fY, fW, fH) = faces
            roi = gray[fY:fY + fH, fX:fX + fW]
            roi = cv2.resize(roi, (64, 64))
            roi = roi.astype("float") / 255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)
            
            preds = emotion_classifier.predict(roi)[0]
            emotion_probability = np.max(preds)
            label = EMOTIONS[preds.argmax()]

            ee = []
            percent = []
            for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
                ee.append(emotion)
                percent.append(prob)
            mp = percent.index(max(percent))

        cv2.imshow('your_face', frameClone)
        break
    camera.release()
    cv2.destroyAllWindows()
    try:
        return ee[mp] 
    except:
        return "I cannot see your face."

Using TensorFlow backend.


# Face Identification Connection

In [4]:
%matplotlib inline

import cv2
import matplotlib.pyplot as plt
from IPython import display

import face_recognition
import glob
users = glob.glob("Users\*.jpg")
# Load a sample picture and learn how to recognize it.
known_face_encodings = []
known_face_names = []

for user in users:
    user_image = face_recognition.load_image_file(user)
    known_face_encodings.append(face_recognition.face_encodings(user_image)[0])
    known_face_names.append(user.split("\\")[1].split(".")[0])
print(known_face_names)

['obama', 'pratik']


In [5]:
import face_recognition

def identifyu(query=0,sessionID="general"):
    video_capture = cv2.VideoCapture(0)

    # Load a sample picture and learn how to recognize it.


    # while True:
        # Grab a single frame of video
    ret, frame = video_capture.read()

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_frame = frame[:, :, ::-1]

    # Find all the faces and face enqcodings in the frame of video
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
    name = "Unknown"
    # Loop through each face in this frame of video
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # See if the face is a match for the known face(s)
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)

        name = "Unknown"

        # If a match was found in known_face_encodings, just use the first one.
        if True in matches:
            first_match_index = matches.index(True)
            name = known_face_names[first_match_index]

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
    #webcam_preview = plt.imshow(frame)

#     # Hit 'q' on the keyboard to quit!
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
#     break

    # Release handle to the webcam
    video_capture.release()
    cv2.destroyAllWindows()
    return name

# Save Mood and Load The User template file

In [66]:
def whathappen(query,sessionID="general"):
    aa = input()
    nam = identifyu()
    with open(nam+".txt", "a") as myfile:
        myfile.write(aa)
    return "Would you like to tell me more about it?"
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
def sas(sentence):
    if analyser.polarity_scores(sentence)['pos']>analyser.polarity_scores(sentence)['neg']:
        return "happy"
    else:
        return "sad"

In [73]:
def chat():
    nam = identifyu()
    call = multiFunctionCall({"whoIs":whoIs,"emo":emo, "identifyu":identifyu, "whathappen":whathappen})
    if nam == "Unknown":
        firstQuestion="Hi, I am chatbot."
        template = "Example.template"
    else :
        firstQuestion="Hi "+nam+" , nice to see you again."
        template = nam+".template"
    Chat(template, reflections,call=call).converse(firstQuestion)
    from os import path
    if path.exists(nam+".txt"):
        with open(nam+".txt", "r") as myfile:
            daa = myfile.read()
            with open("pratik.template", "r") as myfile:
                try:
                    mood = (myfile.readlines()[-2][2]) == "m"
                except:
                    mood = False
            if mood:
                with open(nam+".template", "r+") as f:
                    d = f.readlines()
                    f.seek(0)
                    for i in d[:-2]:
                        f.write(i)
                    f.truncate()
            with open(nam+".template", "a") as myf:
                myf.write("\n{ mood : "+sas(daa)+" }")
                myf.write("\n{ reason : "+daa+" }")
        os.remove(nam+".txt")


In [71]:
chat()

Hi pratik , nice to see you again.
> It is a rough day today
Would you like to talk about it?
> yes
Go ahead, tell me what happend.
> bad time in two and a half men show
bad time in two and a half men show
Would you like to tell me more about it?
> no
no.
> quit
Thank you for talking with me.


In [74]:
with open("pratik.template", "r") as myfile:
    try:
        mood = (myfile.readlines()[-2][2]) == "m"
    except:
        mood = False
if mood:
    with open("pratik.template", "r") as myfile:
        for line in myfile.readlines()[-2:]:
            print(line)

{ mood : sad }

{ reason : bad time in two and a half men show }


# Testing below this point

In [ ]:
Chat("Example.template", reflections,call=call).converse(firstQuestion)

# Text to Voice Code

In [ ]:
import pyttsx3
engine = pyttsx3.init()
voices = engine.getProperty('voices')       #getting details of current voice
engine.setProperty('voice', voices[1].id)
engine.say("I will speak this text")
engine.runAndWait()

In [ ]:
import pyttsx3
engine = pyttsx3.init() # object creation

""" RATE"""
rate = engine.getProperty('rate')   # getting details of current speaking rate
print (rate)                        #printing current voice rate
engine.setProperty('rate', 125)     # setting up new voice rate


"""VOLUME"""
volume = engine.getProperty('volume')   #getting to know current volume level (min=0 and max=1)
print (volume)                          #printing current volume level
engine.setProperty('volume',1.0)    # setting up volume level  between 0 and 1

"""VOICE"""
voices = engine.getProperty('voices')       #getting details of current voice
#engine.setProperty('voice', voices[0].id)  #changing index, changes voices. o for male
engine.setProperty('voice', voices[1].id)   #changing index, changes voices. 1 for female

engine.say("Hello World!")
engine.say('My current speaking rate is ' + str(rate))
engine.runAndWait()
engine.stop()

In [ ]:
import glob
a = glob.glob("Users\*.jpg")

In [ ]:
a

In [ ]:
import face_recognition

In [ ]:
obama_image = face_recognition.load_image_file(a[1])

In [ ]:
obama_face_encoding = face_recognition.face_encodings(obama_image)[0]

In [ ]:
plt.imshow(obama_image)

In [ ]:
import glob
users = glob.glob("Users\*.jpg")
# Load a sample picture and learn how to recognize it.
users_face_encoding = []
users_names = []

for user in users:
    user_image = face_recognition.load_image_file(user)
    users_face_encoding.append(face_recognition.face_encodings(user_image))
    users_names.append(user.split("\\")[1])
    plt.imshow(user_image)
#print(users_names)
    
# obama_image = face_recognition.load_image_file("obama.jpg")
# obama_face_encoding = face_recognition.face_encodings(obama_image)[0]

# # Load a second sample picture and learn how to recognize it.
# biden_image = face_recognition.load_image_file("biden.jpg")
# biden_face_encoding = face_recognition.face_encodings(biden_image)[0]

# # Create arrays of known face encodings and their names
# known_face_encodings = [
#     obama_face_encoding,
#     biden_face_encoding
# ]
# known_face_names = [
#     "Barack Obama",
#     "Pratik Savla"
# ]

In [ ]:
%matplotlib inline

import cv2
import matplotlib.pyplot as plt
from IPython import display

In [ ]:
vc = cv2.VideoCapture(0)

if vc.isOpened(): # try to get the first frame
    is_capturing, frame = vc.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)    # makes the blues image look real colored
    webcam_preview = plt.imshow(frame)    
else:
    is_capturing = False

while is_capturing:
    try:    # Lookout for a keyboardInterrupt to stop the script
        is_capturing, frame = vc.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)    # makes the blues image look real colored
        webcam_preview.set_data(frame)
        plt.draw()

        display.clear_output(wait=True)
        display.display(plt.gcf())

        plt.pause(0.1)    # the pause time is = 1 / framerate
    except KeyboardInterrupt:
        vc.release()
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    break

# Release handle to the webcam
vc.release()
cv2.destroyAllWindows()


In [ ]:
video_capture.release()


In [ ]:
with open("test.txt", "a") as myfile:
    myfile.write(" appended text aaa ")

In [47]:
with open("pratik.template", "r") as myfile:
    try:
        mood = (myfile.readlines()[-2][2]) == "m"
    except:
        mood = False
if mood:
    with open("pratik.template", "r") as myfile:
        print(myfile.readlines()[-2:])
        

In [ ]:
from os import path
path.exists("pratik.txt")

In [ ]:
def chat():
    nam = identifyu()
    call = multiFunctionCall({"whoIs":whoIs,"emo":emo, "identifyu":identifyu, "whathappen":whathappen})
    if nam == "Unknown":
        firstQuestion="Hi, I am chatbot."
        template = "Example.template"
    else :
        firstQuestion="Hi "+nam+" , nice to see you again."
        template = nam+".template"
    Chat(template, reflections,call=call).converse(firstQuestion)
    from os import path
    if path.exists(nam+".txt"):
        with open(nam+".txt", "r") as myfile:
            daa = myfile.read()
            with open(nam+".template", "a") as myf:
                myf.write("\n\n{ mood : "+daa+" }")
        os.remove(nam+".txt")


In [ ]:
def whathappen(query,sessionID="general"):
    aa = input()
    with open("pratik"+".txt", "a") as myfile:
        myfile.write(aa)
    return "Would you like to tell me more about it?"

In [ ]:
chat()

In [ ]:
from datetime import date
today = date.today()
print("Today's date:", today)

In [ ]:
d1 = today.strftime("%d/%m/%Y")
print("d1 =", d1)

In [ ]:
today.strftime("%B")

In [9]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [48]:
def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    print(score)

In [49]:
sentiment_analyzer_scores("It is a good day")

{'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'compound': 0.4404}
